In [1]:
%reload_ext autoreload
%autoreload 2

import nltk
nltk.download('stopwords')
nltk.download('punkt')


#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
!from xgboost import XGBClassifier

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_space.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = bad_symbols.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text 

dataset['Story'] = dataset['Story'].apply(preprocess)
test['Story'] = test['Story'].apply(preprocess)

data_train_list = dataset['Story'].tolist()
data_test_list = test['Story'].tolist()

all_data = data_train_list + data_test_list



tt = WordPunctTokenizer()
count_vect=CountVectorizer(tokenizer = tt.tokenize ,stop_words ='english', ngram_range = (1,3))
count_vect.fit(all_data)
train_countvect = count_vect.transform(dataset.Story)
test_countvect = count_vect.transform(test.Story)

## Feature Selection
#Dimensionality reduction using LSI
svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(train_countvect)
train_countvect_LSI=svd.transform(train_countvect)

svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(test_countvect)
test_countvect_LSI=svd.transform(test_countvect)

#Feature extraction using TFxIDF
tf_idf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1.0,smooth_idf=1.0,sublinear_tf=1.0,
            stop_words = 'english')
tf_idf.fit(all_data)
train_tf= tf_idf.transform(dataset.Story)
test_tf = tf_idf.transform(test.Story)

## Feature Selection
#Dimensionality reduction using LSI
svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(train_tf)
train_tf_LSI=svd.transform(train_tf)

svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(test_tf)
test_tf_LSI=svd.transform(test_tf)

# defining parameter range 
param_grid = {'n_estimators': [10,50,100,200],
     'learning_rate': [0.01, 0.05, 0.1, 1]
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(AdaBoostClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
from: can't read /var/mail/xgboost


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Fitting 1157 folds for each of 16 candidates, totalling 18512 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1564 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2044 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 3868 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 4604 tasks      | elapsed: 37.5min
[Parallel(n_jobs=-1)]: Done 5404 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done 6268 tasks      | elapsed: 41.3min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | e

{'learning_rate': 1, 'n_estimators': 200}
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=200, random_state=None)


In [2]:
best_params = {'n_estimators': 200, 'learning_rate': 1}
ada_clf = AdaBoostClassifier(**best_params)
ada_clf.fit(train_tf_LSI, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=200, random_state=None)